In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

import sys
sys.path.append(r"D:\coding\Jupyter_notebooks\ML")
from credit_risk.read_data import read_data_train
from credit_risk.transform_pipeline import data_train_pipeline

random_seed = 42

In [3]:
data_original = read_data_train()

In [4]:
data_train = data_train_pipeline.fit_transform(data_original.copy(deep=True))
data_train = data_train.astype("float32")

In [5]:
y_columns = ["TARGET"]

In [6]:
from tensorflow import keras
from tensorflow.keras import layers

credit_default_model1 = keras.Sequential([
    keras.Input(shape=(len(data_train.columns) - len(y_columns),), name="input"),
    layers.Dense(200, activation="relu", name="dense_1"),
    layers.Dense(1, activation="sigmoid", name="output")
], name="credit_default_model")

credit_default_model1.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.Adam())

ImportError: Traceback (most recent call last):
  File "c:\users\anton\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 64, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [ ]:
credit_default_model1.summary()

In [7]:
from sklearn.linear_model import LogisticRegressionCV
credit_default_model2 = LogisticRegressionCV()

In [8]:
from sklearn.naive_bayes import ComplementNB
credit_default_model3 = ComplementNB()

In [9]:
from sklearn.ensemble import RandomForestClassifier
credit_default_model4 = RandomForestClassifier(200)

In [ ]:
# tp, fp, fn, tn
# must be 0.333
f1_score(np.array([1, 0, 1, 0, 0, 0]), np.array([1, 1, 0, 0, 1, 1]))

In [ ]:
def train_model(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    show_f1_score(model, x_test, y_test)

def keras_train_model(model, x_train, x_test, y_train, y_test, batch_size=500, epochs=10):
    history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    show_keras_history(history)
    show_f1_score(model, x_test, y_test)

def show_keras_history(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.show()

In [ ]:
keras_train_model(credit_default_model1, *split_train_data(data_train), 500, 10)

In [ ]:
train_model(credit_default_model2, *split_train_data(data_train))

In [ ]:
train_model(credit_default_model3, *split_train_data(data_train))


In [ ]:
train_model(credit_default_model4, *split_train_data(data_train))


In [ ]:
from credit_risk.transform_pipeline import UpsampleByTarget

x_tr, x_ts, y_tr, y_ts = split_train_data(data_train)
data_upsampled = UpsampleByTarget("TARGET").transform(pd.concat([x_tr, y_tr], axis="columns"))

x_tr_upsampled, y_tr_upsampled = get_data_xy(data_upsampled)

In [ ]:
sn.countplot(x="TARGET", data=y_tr_upsampled)

In [ ]:
sn.countplot(x="TARGET", data=y_ts)

In [ ]:
# test with upsampling
keras_train_model(credit_default_model1, x_tr_upsampled, x_ts, y_tr_upsampled, y_ts, epochs=10)

In [ ]:
train_model(credit_default_model2, x_tr_upsampled, x_ts, y_tr_upsampled, y_ts)

In [ ]:
train_model(credit_default_model3, x_tr_upsampled, x_ts, y_tr_upsampled, y_ts)

In [ ]:
train_model(credit_default_model4, x_tr_upsampled, x_ts, y_tr_upsampled, np.asarray(y_ts).reshape((len(y_ts),)))


In [ ]:
show_f1_score(credit_default_model3, *get_data_xy(data_train))